# The Aries Basic Controller

The aries basic controller is a simple python library that wraps a swagger-api exposed by the [ACA-Py Agent implementation](https://github.com/hyperledger/aries-cloudagent-python). It is designed to enable easy, programatic interface between business logic of an application (this is what you will write) and the capabilities of the Hyperledger SSI stack currently implemented by this agent codebase as a series of protocols. These are all defined in [aries-rfcs](https://github.com/hyperledger/aries-rfcs).

## The ACA-Py Swagger-API

As we discussed in the [mental model for ssi application development](https://hackmd.io/YuQugi8dS0eAVuT1T1vaug) section of this course, aries agents are need to be controller. They understand a series of protocols and their associated messages which they can send and receive to other agents, but they do not know when to send them or what to do if they receive them.

ACA-Py agents expose a swagger-api, defining a set of api endpoints you can use to communicate with an agent controlling it to create and send messages to other agents and they interact in defined aries protocols. You should be able to see the swagger documentation at this endpoint - http://localhost:8021/api/doc. Note if you are running this on a virtual machine such as an AWS EC2 instance you will need replace localhost with the IP address for that machine.

Throughout this tutorial series you will be controlling the agents we have defined using this api. If you want to dive deeper into aries I highly recommend taking the time to go through this lab [OpenApi lab](https://github.com/cloudcompass/ToIPLabs/blob/master/docs/LFS173x/OpenAPIController.md) from the an [EdX course](https://www.edx.org/course/becoming-a-hyperledger-aries-developer).

Instead of using the swagger interface to interact with the agents we will be using the aries-basic-controller. 


## Aries Basic Controller Library Structure

The library is fairly straightforward. If you have a look into the [aries_basic_controller folder](https://github.com/OpenMined/PyDentity/tree/master/libs/aries-basic-controller/aries_basic_controller). You will see all the code for the library.

The most important file is `aries_controller.py`, this contains the AriesAgentController class which is the class you will import and instatiate to control a specific ACA-Py instance.

The other files are contained withing three folders:

* **controllers/** - This has a file for each collection of api endpoints. As well as a base.py file which all other controllers extend handling the Restful logic through aiohttp.
* **models/** - This currently only contains connection.py and is used to enable the controller to await a certain state change in a connection. More on this later.
* **helpers/** - Some basic helper functions.

## Instantiating the a AriesAgentController

In [ ]:
# Import the AriesAgentController class from the library
from aries_basic_controller.aries_controller import AriesAgentController

# Define the parameters, these should be the same ones uses to initalise the ACA-Py Agent

# The location the controller spins up a service and listens for webhooks from the agent
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
# Configures the controller with the correct endpoint to send api requests too
ADMIN_URL = "http://om-authority-agent:8021"



# Instantiate the AriesAgentController using the correct parameters
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

## Understanding the Aries Basic Controller Listeners

An instance of AriesAgentController acts as the middle man between the business logic of an application and an ACA-Py agent instance. The business logic can communicate with the ACA-Py instance by sending specific requests to the ADMIN_URL, these requests are implemented in the controllers folder of the aries-basic-controller library.

However, the ACA-Py agent instance also needs to be able to communicate to the business logic of the application. When the agent recieves a message from another agent over a connection, the business logic needs to be able to interpret that message and if needs be respond by communicating through the controller to the agent to take some action.

This is handled through ACA-Py using webhooks. Whenever an ACA-Py agent receives a message, if webhooks are configured (--webhook-url), then generally they post a webhook event identified through the key word topic to the specified endpoint. 

For an instance of AriesBasicController to be listening for these webhook events you must initialise the server by calling:

In [ ]:
await agent_controller.listen_webhooks()

This means that the controller is receiving api requests sent to this endpoint and calling the recieve_webhook function:

```app.add_routes([web.post(self.webhook_base + "/topic/{topic}/", self._receive_webhook)])```

The controller then uses the [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/) library to emit events for every webhook they receive:

```    async def handle_webhook(self, topic, payload):
        logging.debug(f"Handle Webhook - {topic}", payload)
        pub.sendMessage(topic, payload=payload)
        return web.Response(status=200)```

It is up to the business logic of the application (*YOU THE DEVELOPER*) to register listeners for the topic events that your application is interested in and the handler functions specifying how the application will respond to these events.

You can register listeners like this:

**Note: Topics are all defined by ACA-Py. Generally there is a topic for each protocol**

In [ ]:
def some_handler(payload):
    print("WRITE YOUR BUSINESS LOGIC HERE")

## The topic must match the topic used by the agent for the webhook.
some_listener = {
    "handler": some_handler,
    "topic": "sometopic"
}

## You can add as many listeners into this array as you want
## The basic controller also specifies some optional default listeners 
## which help update state (see controllers/connection.py) and have some helpful print statements
agent_controller.register_listeners([some_listener], defaults=True)

### You can also add listeners one at a time.

And add more than one listener for the same topic. This is useful in larger applications where multiple processes need to react to a trigger from the agent.

![PyPubSub figure](https://pypubsub.readthedocs.io/en/v4.0.3/_images/pubsub_concept.png)

In [1]:
def some_handler2(payload):

    print("listener 2")
    # Some other custom function

    
some_listener2 = {
    "handler": some_handler2,
    "topic": "sometopic"
}

agent_controller.add_listener(some_listener2)

NameError: name 'agent_controller' is not defined

### Listeners can be removed individually

In [ ]:
agent_controller.remove_listener(some_listener2)

### And by topic

If topic == None then all listeners will be removed.

In [ ]:
topic = "sometopic"
agent_controller.remove_all(topic)



## Terminating the Controller

At the end of all notebook the controller instance must be terminated, so that a new one can be created in following tutorials. If you run into errors around port already in use, you likely forgot to terminate a controller instance from a previous tutorial.

In [ ]:
await agent_controller.terminate()

# Continue to Part 3

Great, now you should have a better idea about what the aries-basic-controller actually is. Throughout the rest of these notebooks we will be using this package to facillitate common interaction flows between the different agents in our scenario.

First up is how to initialise an agent as an issuing authority on the public ledger.

